# Analysis for China wage vs. education data

#### Since R cannot be ran on normal VS code, I had to install visual studio code insider for it!

## Datasets used

From Dropbox: China-Education-Wages -> Data -> CFPS Data 2010-2016

Downloaded:
- 2010 Egnlish -> ecfps2010adult_112014.dta     
  - Renamed 2010adult.dta
- 2012 Egnlish -> ecfps2012adultcombine…015.dta     
  - Renamed 2012adult.dta
- 2014 English -> ecfps2014adult_170630.dta     
  - Renamed 2014adult.dta
- Mincer16 -> Mincer16.csv 

All 4 datasets were put within one folder named $CFPSdata$ in the same layer as this jupyter notebook.

In [1]:
library(knitr)
library(xtable)
library(broom)
library(dplyr)
library(tidyverse)
library(ggplot2)
library(stargazer)
library(lubridate)
library(haven)
library(ineq)
library(PerformanceAnalytics)
library(gglorenz)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.2     v purrr   0.3.4
v tibble  3.0.4     v stringr 1.4.0
v tidyr   1.1.2     v forcats 0.5.0
v readr   1.4.0     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: xts

Loading required package: zoo


Attaching package: 'zoo'


The following objects are

## Manipulating data

### Read Data

In [2]:
mincer10 <- read_dta("./CFPSdata/mincer10.dta")
mincer12 <- read_dta("./CFPSdata/mincer12.dta")
mincer14 <- read_dta("./CFPSdata/mincer14.dta")
mincer16 <- read_dta("./CFPSdata/mincer16.dta")

### Adding education and year dummies to the data, then combining them into one dataframe

In [3]:
pinc <- function(df) {
    categorize <-
        df %>%
        group_by(rinc) %>%
        tally()
    categorize <- categorize[order(categorize$rinc), ]
    sum_so_far <- 0
    total <- nrow(df)
    categorize$pinc <- 0
    for (row in seq_len(nrow(categorize))) {
      sum_so_far <- sum_so_far + categorize[row, "n"]
      categorize[row, "pinc"] <- (sum_so_far -
       (0.5 * categorize[row, "n"])) / total
    }
    ordered_combined <- df[order(df$rinc), ]
    ordered_combined$pinc <- 0
    current_rinc <- 0
    current_pinc <- 0
    current_row <- 0
    for (row in seq_len(nrow(ordered_combined))) {
      rinc <- ordered_combined[row, "rinc"]
      if (rinc != current_rinc) {
        current_row <- current_row + 1
        current_rinc <- rinc
        current_pinc <- categorize[current_row, "pinc"]
      }
      ordered_combined[row, "pinc"] <- current_pinc
    }
    ordered_combined$lnpinc <- log(ordered_combined$pinc)
    return(ordered_combined)
}

In [4]:
mincer10_urban <- filter(mincer10, urban == 1)
mincer10_rural <- filter(mincer10, urban == 0)
mincer10_female_urban <- filter(mincer10_urban, gender == 1)
mincer10_female_rural <- filter(mincer10_rural, gender == 1)
mincer10_male_urban <- filter(mincer10_urban, gender == 0)
mincer10_male_rural <- filter(mincer10_rural, gender == 0)

mincer10$rinc <- mincer10$income / mean(mincer10$income)

mincer10_urban$rinc <- mincer10_urban$income / mean(mincer10_urban$income)
mincer10_rural$rinc <- mincer10_rural$income / mean(mincer10_rural$income)
mincer10_female_urban$rinc <- mincer10_female_urban$income /
 mean(mincer10_female_urban$income)
mincer10_female_rural$rinc <- mincer10_female_rural$income /
 mean(mincer10_female_rural$income)
mincer10_male_urban$rinc <- mincer10_male_urban$income /
 mean(mincer10_male_urban$income)
mincer10_male_rural$rinc <- mincer10_male_rural$income /
 mean(mincer10_male_rural$income)

In [5]:
mincer10 <- pinc(mincer10)

mincer10_urban <- pinc(mincer10_urban)

mincer10_rural <- pinc(mincer10_rural)

mincer10_female_urban <- pinc(mincer10_female_urban)

mincer10_female_rural <- pinc(mincer10_female_rural)

mincer10_male_urban <- pinc(mincer10_male_urban)

mincer10_male_rural <- pinc(mincer10_male_rural)

In [6]:
mincer12_urban <- filter(mincer12, urban == 1)
mincer12_rural <- filter(mincer12, urban == 0)
mincer12_female_urban <- filter(mincer12_urban, gender == 1)
mincer12_female_rural <- filter(mincer12_rural, gender == 1)
mincer12_male_urban <- filter(mincer12_urban, gender == 0)
mincer12_male_rural <- filter(mincer12_rural, gender == 0)

mincer12$rinc <- mincer12$income / mean(mincer12$income)

mincer12_urban$rinc <- mincer12_urban$income / mean(mincer12_urban$income)
mincer12_rural$rinc <- mincer12_rural$income / mean(mincer12_rural$income)
mincer12_female_urban$rinc <- mincer12_female_urban$income /
 mean(mincer12_female_urban$income)
mincer12_female_rural$rinc <- mincer12_female_rural$income /
 mean(mincer12_female_rural$income)
mincer12_male_urban$rinc <- mincer12_male_urban$income /
 mean(mincer12_male_urban$income)
mincer12_male_rural$rinc <- mincer12_male_rural$income /
 mean(mincer12_male_rural$income)

In [7]:
mincer12 <- pinc(mincer12)

mincer12_urban <- pinc(mincer12_urban)

mincer12_rural <- pinc(mincer12_rural)

mincer12_female_urban <- pinc(mincer12_female_urban)

mincer12_female_rural <- pinc(mincer12_female_rural)

mincer12_male_urban <- pinc(mincer12_male_urban)

mincer12_male_rural <- pinc(mincer12_male_rural)

In [8]:
mincer14_urban <- filter(mincer14, urban == 1)
mincer14_rural <- filter(mincer14, urban == 0)
mincer14_female_urban <- filter(mincer14_urban, gender == 1)
mincer14_female_rural <- filter(mincer14_rural, gender == 1)
mincer14_male_urban <- filter(mincer14_urban, gender == 0)
mincer14_male_rural <- filter(mincer14_rural, gender == 0)

mincer14$rinc <- mincer14$income / mean(mincer14$income)

mincer14_urban$rinc <- mincer14_urban$income / mean(mincer14_urban$income)
mincer14_rural$rinc <- mincer14_rural$income / mean(mincer14_rural$income)
mincer14_female_urban$rinc <- mincer14_female_urban$income /
 mean(mincer14_female_urban$income)
mincer14_female_rural$rinc <- mincer14_female_rural$income /
 mean(mincer14_female_rural$income)
mincer14_male_urban$rinc <- mincer14_male_urban$income /
 mean(mincer14_male_urban$income)
mincer14_male_rural$rinc <- mincer14_male_rural$income /
 mean(mincer14_male_rural$income)

In [9]:
mincer14 <- pinc(mincer14)

mincer14_urban <- pinc(mincer14_urban)

mincer14_rural <- pinc(mincer14_rural)

mincer14_female_urban <- pinc(mincer14_female_urban)

mincer14_female_rural <- pinc(mincer14_female_rural)

mincer14_male_urban <- pinc(mincer14_male_urban)

mincer14_male_rural <- pinc(mincer14_male_rural)

In [10]:
mincer16_urban <- filter(mincer16, urban == 1)
mincer16_rural <- filter(mincer16, urban == 0)
mincer16_female_urban <- filter(mincer16_urban, gender == 1)
mincer16_female_rural <- filter(mincer16_rural, gender == 1)
mincer16_male_urban <- filter(mincer16_urban, gender == 0)
mincer16_male_rural <- filter(mincer16_rural, gender == 0)

mincer16$rinc <- mincer16$income / mean(mincer16$income)

mincer16_urban$rinc <- mincer16_urban$income / mean(mincer16_urban$income)
mincer16_rural$rinc <- mincer16_rural$income / mean(mincer16_rural$income)
mincer16_female_urban$rinc <- mincer16_female_urban$income /
 mean(mincer16_female_urban$income)
mincer16_female_rural$rinc <- mincer16_female_rural$income /
 mean(mincer16_female_rural$income)
mincer16_male_urban$rinc <- mincer16_male_urban$income /
 mean(mincer16_male_urban$income)
mincer16_male_rural$rinc <- mincer16_male_rural$income /
 mean(mincer16_male_rural$income)

In [11]:
mincer16 <- pinc(mincer16)

mincer16_urban <- pinc(mincer16_urban)

mincer16_rural <- pinc(mincer16_rural)

mincer16_female_urban <- pinc(mincer16_female_urban)

mincer16_female_rural <- pinc(mincer16_female_rural)

mincer16_male_urban <- pinc(mincer16_male_urban)

mincer16_male_rural <- pinc(mincer16_male_rural)

In [12]:
combined <- full_join(mincer10,
 full_join(mincer12, full_join(mincer14, mincer16)))
combined$lninc <- log(combined$income)
combined$lnrinc <- log(combined$rinc)

urban <- full_join(mincer10_urban,
 full_join(mincer12_urban, full_join(mincer14_urban, mincer16_urban)))
urban$lninc <- log(urban$income)
urban$lnrinc <- log(urban$rinc)


rural <- full_join(mincer10_rural,
 full_join(mincer12_rural, full_join(mincer14_rural, mincer16_rural)))
rural$lninc <- log(rural$income)
rural$lnrinc <- log(rural$rinc)


female_urban <- full_join(mincer10_female_urban,
 full_join(mincer12_female_urban,
 full_join(mincer14_female_urban, mincer16_female_urban)))
female_urban$lninc <- log(female_urban$income)
female_urban$lnrinc <- log(female_urban$rinc)


female_rural <- full_join(mincer10_female_rural,
 full_join(mincer12_female_rural,
 full_join(mincer14_female_rural, mincer16_female_rural)))
female_rural$lninc <- log(female_rural$income)
female_rural$lnrinc <- log(female_rural$rinc)


male_urban <- full_join(mincer10_male_urban,
 full_join(mincer12_male_urban,
 full_join(mincer14_male_urban, mincer16_male_urban)))
male_urban$lninc <- log(male_urban$income)
male_urban$lnrinc <- log(male_urban$rinc)

male_rural <- full_join(mincer10_male_rural,
 full_join(mincer12_male_rural,
 full_join(mincer14_male_rural, mincer16_male_rural)))
male_rural$lninc <- log(male_rural$income)
male_rural$lnrinc <- log(male_rural$rinc)

Joining, by = c("pid", "income", "age", "gender", "urban", "prov", "ethnic", "married", "party", "postsecondary", "seniorsecondary", "juniorsecondary", "primary", "illiterate", "y10", "y12", "y14", "y16", "edu", "Beijing", "Tianjin", "Hebei", "Shanxi", "Liaoning", "Jilin", "Heilongjiang", "Shanghai", "Jiangsu", "Zhejiang", "Anhui", "Fujian", "Jiangxi", "Shandong", "Henan", "Hubei", "Hunan", "Guangdong", "Guangxi_Zhuang_Autonomous_Region", "Chongqing", "Sichuan", "Guizhou", "Yunnan", "Shaanxi", "Gansu", "rinc", "pinc", "lnpinc")

Joining, by = c("pid", "income", "age", "gender", "urban", "prov", "ethnic", "married", "party", "postsecondary", "seniorsecondary", "juniorsecondary", "primary", "illiterate", "y10", "y12", "y14", "y16", "edu", "Beijing", "Tianjin", "Hebei", "Shanxi", "Liaoning", "Jilin", "Heilongjiang", "Shanghai", "Jiangsu", "Zhejiang", "Anhui", "Fujian", "Jiangxi", "Shandong", "Henan", "Hubei", "Hunan", "Guangdong", "Guangxi_Zhuang_Autonomous_Region", "Chongqing", "Sichuan"

### Computing Lorenz curves for combined, male, and female for each year

In [ ]:
lorenz_combined <- filter(combined, income >= 0)

lorenz_combined$year_str[lorenz_combined$y10 == 1] <- "Year 2010"
lorenz_combined$year_str[lorenz_combined$y12 == 1] <- "Year 2012"
lorenz_combined$year_str[lorenz_combined$y14 == 1] <- "Year 2014"
lorenz_combined$year_str[lorenz_combined$y16 == 1] <- "Year 2016"

lorenz_male <- filter(lorenz_combined, gender == 0)
lorenz_female <- filter(lorenz_combined, gender == 1)

ggplot(lorenz_combined, aes(x = income, colour = year_str)) +
  stat_lorenz() +
  geom_abline(color = "grey") +
  labs(x = "Cumulative percentages of all income",
       y = "Cumulative percentage of percentage of people with that income",
       title = "Lorenz curves for combined data")

ggsave("LorenzCurves/Lorenz_curve_combined.png")

ggplot(lorenz_male, aes(x = income, colour = year_str)) +
  stat_lorenz() +
  geom_abline(color = "grey") +
  labs(x = "Cumulative percentages of all income",
       y = "Cumulative percentage of percentage of people with that income",
       title = "Lorenz curves for male data")

ggsave("LorenzCurves/Lorenz_curve_male.png")

ggplot(lorenz_female, aes(x = income, colour = year_str)) +
  stat_lorenz() +
  geom_abline(color = "grey") +
  labs(x = "Cumulative percentages of all income",
       y = "Cumulative percentage of percentage of people with that income",
       title = "Lorenz curves for female data")

ggsave("LorenzCurves/Lorenz_curve_female.png")

In [16]:
sprintf("Combined: %f, %f, %f, %f", Gini(mincer10$income),
 Gini(mincer12$income), Gini(mincer14$income), Gini(mincer16$income))
fem_10 <- filter(mincer10, gender == 1)
fem_12 <- filter(mincer12, gender == 1)
fem_14 <- filter(mincer14, gender == 1)
fem_16 <- filter(mincer16, gender == 1)
male_10 <- filter(mincer10, gender == 0)
male_12 <- filter(mincer12, gender == 0)
male_14 <- filter(mincer14, gender == 0)
male_16 <- filter(mincer16, gender == 0)
sprintf("Female: %f, %f, %f, %f", Gini(fem_10$income),
 Gini(fem_12$income), Gini(fem_14$income), Gini(fem_16$income))
 sprintf("Male: %f, %f, %f, %f", Gini(male_10$income),
 Gini(male_12$income), Gini(male_14$income), Gini(male_16$income))
#sprintf("Urban: %f, %f, %f, %f", Gini(mincer10_urban$income),
#Gini(mincer12_urban$income),
#Gini(mincer14_urban$inc), Gini(mincer16_urban$inc))
#sprintf("Rural: %f, %f, %f, %f", Gini(mincer10_rural$inc),
#Gini(mincer12_rural$inc), Gini(mincer14_rural$inc), Gini(mincer16_rural$inc))
#sprintf("Female Urban: %f, %f, %f, %f",
#Gini(mincer10_female_urban$inc), Gini(mincer12_female_urban$inc),
#Gini(mincer14_female_urban$inc), Gini(mincer16_female_urban$inc))
#sprintf("Female Rural: %f, %f, %f, %f",
#Gini(mincer10_female_rural$inc), Gini(mincer12_female_rural$inc),
#Gini(mincer14_female_rural$inc), Gini(mincer16_female_rural$inc))
#sprintf("Male Urban: %f, %f, %f, %f",
 #Gini(mincer10_male_urban$inc), Gini(mincer12_male_urban$inc),
  #Gini(mincer14_male_urban$inc), Gini(mincer16_male_urban$inc))
#sprintf("male Rural: %f, %f, %f, %f",
 #Gini(mincer10_male_rural$inc), Gini(mincer12_male_rural$inc),
  #Gini(mincer14_male_rural$inc), Gini(mincer16_male_rural$inc))

[1] "Combined: 0.582171, 0.477763, 0.623096, 0.536153"

[1] "Female: 0.548434, 0.464359, 0.584124, 0.539832"

[1] "Male: 0.608647, 0.479520, 0.662954, 0.505255"

## Running the descriptive analysis for the all of the sub populations

### For combined data

In [ ]:
hist(combined$lninc)
hist(urban$lninc)
hist(rural$lninc)
hist(female_urban$lninc)
hist(female_rural$lninc)
hist(male_urban$lninc)
hist(male_rural$lninc)


In [ ]:
hist(combined$lnrinc)
hist(urban$lnrinc)
hist(rural$lnrinc)
hist(female_urban$lnrinc)
hist(female_rural$lnrinc)
hist(male_urban$lnrinc)
hist(male_rural$lnrinc)

In [ ]:
hist(combined$lnpinc)
hist(urban$lnpinc)
hist(rural$lnpinc)
hist(female_urban$lnpinc)
hist(female_rural$lnpinc)
hist(male_urban$lnpinc)
hist(male_rural$lnpinc)

## Running the regressions for lninc, rinc, lnrinc, pinc, and lnpinc on combined data, and urban and rural subsamples. Does not contain married or party variables

### Finding which dummies to use

In [13]:
means <- c()
for (col in c(11, 12, 13, 14, 21, 22, 23, 31, 32, 33, 34, 35,
 36, 37, 41, 42, 43, 44, 45, 50, 51, 52, 53, 61, 62)) {
    temp <- filter(combined, prov == col)
    means <- append(means, mean(temp$income))
}
print(median(means))

[1] 15583.68


With trial, this turned out to be the Shanxi province

### Regression for lninc, without province dummies

In [14]:
lninc_reg <- filter(combined, !is.infinite(lninc) & !is.na(lninc))
combined_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + urban
  + y10 + y12 + y14, data = lninc_reg)

urban_reg <- filter(urban, !is.infinite(lninc) & !is.na(lninc))
urban_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14, data = urban_reg)

rural_reg <- filter(rural, !is.infinite(lninc) & !is.na(lninc))
rural_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14, data = rural_reg)

female_urban_reg <- filter(female_urban, !is.infinite(lninc) & !is.na(lninc))
female_rural_reg <- filter(female_rural, !is.infinite(lninc) & !is.na(lninc))
male_urban_reg <- filter(male_urban, !is.infinite(lninc) & !is.na(lninc))
male_rural_reg <- filter(male_rural, !is.infinite(lninc) & !is.na(lninc))

female_urban_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary
 + y10 + y12 + y14, data = female_urban_reg)
female_rural_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary
 + y10 + y12 + y14, data = female_rural_reg)
male_urban_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary
 + y10 + y12 + y14, data = male_urban_reg)
male_rural_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary
 + y10 + y12 + y14, data = male_rural_reg)

### Regression for lninc, with province dummies

In [15]:
lninc_reg <- filter(combined, !is.infinite(lninc) & !is.na(lninc))

combined_regional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = lninc_reg)

urban_reg <- filter(urban, !is.infinite(lninc) & !is.na(lninc))
urban_regional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = urban_reg)

rural_reg <- filter(rural, !is.infinite(lninc) & !is.na(lninc))
rural_regional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = rural_reg)

female_urban_reg <- filter(female_urban, !is.infinite(lninc) & !is.na(lninc))
female_rural_reg <- filter(female_rural, !is.infinite(lninc) & !is.na(lninc))
male_urban_reg <- filter(male_urban, !is.infinite(lninc) & !is.na(lninc))
male_rural_reg <- filter(male_rural, !is.infinite(lninc) & !is.na(lninc))

female_urban_regional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = female_urban_reg)

female_rural_regional <- lm(lninc ~  postsecondary + seniorsecondary
 + juniorsecondary + primary + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = female_rural_reg)

male_urban_regional <- lm(lninc ~  postsecondary + seniorsecondary
 + juniorsecondary + primary + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = male_urban_reg)

male_rural_regional <- lm(lninc ~  postsecondary + seniorsecondary
 + juniorsecondary + primary + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = male_rural_reg)

In [22]:
stargazer(combined_nonregional, combined_regional,
 type = "text", out = "Regression outputs/Stargazer outputs/lninc_combined.txt")

stargazer(urban_nonregional, urban_regional,
 type = "text", out = "Regression outputs/Stargazer outputs/lninc_urban.txt")

stargazer(rural_nonregional, rural_regional,
 type = "text", out = "Regression outputs/Stargazer outputs/lninc_rural.txt")

stargazer(female_urban_nonregional, female_urban_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lninc_female_urban.txt")

stargazer(female_rural_nonregional, female_rural_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lninc_female_rural.txt")

stargazer(male_urban_nonregional, male_urban_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lninc_male_urban.txt")
 
stargazer(male_rural_nonregional, male_rural_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lninc_male_rural.txt")


                                                   Dependent variable:                   
                                 --------------------------------------------------------
                                                          lninc                          
                                             (1)                          (2)            
-----------------------------------------------------------------------------------------
postsecondary                              2.285***                    2.151***          
                                           (0.025)                      (0.025)          
                                                                                         
seniorsecondary                            1.700***                    1.616***          
                                           (0.023)                      (0.023)          
                                                                                         
juniorsec

In [83]:
index <- c("combined", "urban", "rural", "female_urban",
 "female_rural", "male_urban", "male_rural")
mean_lninc <- c(mean(lninc_reg$lninc), mean(urban_reg$lninc),
 mean(rural_reg$lninc), mean(female_urban_reg$lninc),
 mean(female_rural_reg$lninc), mean(male_urban_reg$lninc),
 mean(male_rural_reg$lninc))

median_lninc <- c(median(lninc_reg$lninc), median(urban_reg$lninc),
 median(rural_reg$lninc), median(female_urban_reg$lninc),
 median(female_rural_reg$lninc), median(male_urban_reg$lninc),
 median(male_rural_reg$lninc))

var_lninc <- c(var(lninc_reg$lninc), var(urban_reg$lninc),
 var(rural_reg$lninc), var(female_urban_reg$lninc),
 var(female_rural_reg$lninc), var(male_urban_reg$lninc),
 var(male_rural_reg$lninc))

kurtosis_lninc <- c(kurtosis(lninc_reg$lninc), kurtosis(urban_reg$lninc),
 kurtosis(rural_reg$lninc), kurtosis(female_urban_reg$lninc),
 kurtosis(female_rural_reg$lninc), kurtosis(male_urban_reg$lninc),
 kurtosis(male_rural_reg$lninc))

skew_lninc <- c(skewness(lninc_reg$lninc), skewness(urban_reg$lninc),
 skewness(rural_reg$lninc), skewness(female_urban_reg$lninc),
 skewness(female_rural_reg$lninc), skewness(male_urban_reg$lninc),
 skewness(male_rural_reg$lninc))

lninc <- data.frame("subsamples" = index, "mean" = mean_lninc,
 "median" = median_lninc, "var" = var_lninc,
 "kurtosis" = kurtosis_lninc, "skew" = skew_lninc)
lninc$stdev <- sqrt(lninc$var)

### Regression for rinc and lnrinc, without province dummies

In [30]:
combined_nonregional_rinc <- lm(rinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + urban
 + y10 + y12 + y14, data = combined)

urban_nonregional_rinc <- lm(rinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + urban
 + y10 + y12 + y14, data = urban)

rural_nonregional_rinc <- lm(rinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + urban
 + y10 + y12 + y14, data = rural)

lnrinc_reg <- filter(combined, !is.infinite(lnrinc) & !is.na(lnrinc))
urban_reg <- filter(urban, !is.infinite(lnrinc) & !is.na(lnrinc))
rural_reg <- filter(rural, !is.infinite(lnrinc) & !is.na(lnrinc))
combined_nonregional <- lm(lnrinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + urban
  + y10 + y12 + y14, data = lninc_reg)

urban_nonregional <- lm(lnrinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14, data = urban_reg)

rural_nonregional <- lm(lnrinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14, data = rural_reg)


female_urban_reg <- filter(female_urban, !is.infinite(lnrinc) & !is.na(lnrinc))
female_rural_reg <- filter(female_rural, !is.infinite(lnrinc) & !is.na(lnrinc))
male_urban_reg <- filter(male_urban, !is.infinite(lnrinc) & !is.na(lnrinc))
male_rural_reg <- filter(male_rural, !is.infinite(lnrinc) & !is.na(lnrinc))


female_urban_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary
 + y10 + y12 + y14, data = female_urban_reg)
female_rural_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary
 + y10 + y12 + y14, data = female_rural_reg)
male_urban_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary
 + y10 + y12 + y14, data = male_urban_reg)
male_rural_nonregional <- lm(lninc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary
 + y10 + y12 + y14, data = male_rural_reg)

In [86]:
index <- c("combined", "urban", "rural", "female_urban",
 "female_rural", "male_urban", "male_rural")
mean_lnrinc <- c(mean(lnrincReg$lnrinc), mean(urban_reg$lnrinc),
 mean(rural_reg$lnrinc), mean(female_urban_reg$lnrinc),
 mean(female_rural_reg$lnrinc), mean(male_urban_reg$lnrinc),
 mean(male_rural_reg$lnrinc))

median_lnrinc <- c(median(lnrincReg$lnrinc), median(urban_reg$lnrinc),
 median(rural_reg$lnrinc), median(female_urban_reg$lnrinc),
 median(female_rural_reg$lnrinc), median(male_urban_reg$lnrinc),
 median(male_rural_reg$lnrinc))

var_lnrinc <- c(var(lnincReg$lnrinc), var(urban_reg$lnrinc),
 var(rural_reg$lnrinc), var(female_urban_reg$lnrinc),
 var(female_rural_reg$lnrinc), var(male_urban_reg$lnrinc),
 var(male_rural_reg$lnrinc))

kurtosis_lnrinc <- c(kurtosis(lnrincReg$lnrinc), kurtosis(urban_reg$lnrinc),
 kurtosis(rural_reg$lnrinc), kurtosis(female_urban_reg$lnrinc),
 kurtosis(female_rural_reg$lnrinc), kurtosis(male_urban_reg$lnrinc),
 kurtosis(male_rural_reg$lnrinc))

skew_lnrinc <- c(skewness(lnrincReg$lnrinc), skewness(urban_reg$lnrinc),
 skewness(rural_reg$lnrinc), skewness(female_urban_reg$lnrinc),
 skewness(female_rural_reg$lnrinc), skewness(male_urban_reg$lnrinc),
 skewness(male_rural_reg$lnrinc))

lnrinc <- data.frame("subsamples" = index, "mean" = mean_lnrinc,
 "median" = median_lnrinc, "var" = var_lnrinc, "kurtosis" = kurtosis_lnrinc,
 "skew" = skew_lnrinc)
lnrinc$stdev <- sqrt(lnrinc$var)

In [ ]:
print("Lnrinc descriptive statistics")
print(lnrinc)

### Regression for rinc and lnrinc, with province dummies

In [31]:
combined_regional_rinc <- lm(rinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = lnrinc_reg)

urban_regional_rinc <- lm(rinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = urban_reg)

rural_regional_rinc <- lm(rinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = rural_reg)

combined_regional <- lm(lnrinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = lnrinc_reg)

urban_regional <- lm(lnrinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = urban_reg)

rural_regional <- lm(lnrinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + gender + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = rural_reg)

female_urban_regional <- lm(lnrinc ~ postsecondary + seniorsecondary
 + juniorsecondary + primary + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = female_urban_reg)

female_rural_regional <- lm(lnrinc ~  postsecondary + seniorsecondary
 + juniorsecondary + primary + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = female_rural_reg)

male_urban_regional <- lm(lnrinc ~  postsecondary + seniorsecondary
 + juniorsecondary + primary + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = male_urban_reg)

male_rural_regional <- lm(lnrinc ~  postsecondary + seniorsecondary
 + juniorsecondary + primary + y10 + y12 + y14
 + Beijing + Tianjin + Hebei + Liaoning + Jilin + Heilongjiang
 + Shanghai + Jiangsu + Zhejiang + Anhui + Fujian + Jiangxi
 + Shandong + Henan + Hubei + Hunan + Guangdong
 + Guangxi_Zhuang_Autonomous_Region + Chongqing + Sichuan
 + Guizhou + Yunnan + Shaanxi + Gansu, data = male_rural_reg)

In [32]:
stargazer(combined_nonregional_rinc, combined_regional_rinc, type = "text",
 out = "Regression outputs/Stargazer outputs/rinc_combined.txt")

stargazer(urban_nonregional_rinc, urban_regional_rinc,
 type = "text", out = "Regression outputs/Stargazer outputs/rinc_urban.txt")

stargazer(rural_nonregional_rinc, rural_regional_rinc,
 type = "text", out = "Regression outputs/Stargazer outputs/rinc_rural.txt")

stargazer(combined_nonregional, combined_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lnrinc_combined.txt")

stargazer(urban_nonregional, urban_regional,
 type = "text", out = "Regression outputs/Stargazer outputs/lnrinc_urban.txt")

stargazer(rural_nonregional, rural_regional,
 type = "text", out = "Regression outputs/Stargazer outputs/lnrinc_rural.txt")

stargazer(female_urban_nonregional, female_urban_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lnrinc_female_urban.txt")

stargazer(female_rural_nonregional, female_rural_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lnrinc_female_rural.txt")

stargazer(male_urban_nonregional, male_urban_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lnrinc_male_urban.txt")
 
stargazer(male_rural_nonregional, male_rural_regional, type = "text",
 out = "Regression outputs/Stargazer outputs/lnrinc_male_rural.txt")


                                                  Dependent variable:                  
                                 ------------------------------------------------------
                                                          rinc                         
                                            (1)                         (2)            
---------------------------------------------------------------------------------------
postsecondary                             1.709***                   1.594***          
                                          (0.025)                     (0.025)          
                                                                                       
seniorsecondary                           0.817***                   0.748***          
                                          (0.022)                     (0.022)          
                                                                                       
juniorsecondary                

### Regression for pinc and lnpinc, without province dummies

In [101]:
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary
 + primary + gender + urban + y10 + y12 + y14, data = combined))
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary
 + primary + gender + y10 + y12 + y14, data = urban))
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary
 + primary + gender + y10 + y12 + y14, data = rural))

lnpincReg <- filter(combined, !is.infinite(lnpinc) & !is.na(lnpinc))
urban_reg <- filter(urban, !is.infinite(lnpinc) & !is.na(lnpinc))
rural_reg <- filter(rural, !is.infinite(lnpinc) & !is.na(lnpinc))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + urban + y10 + y12 + y14, data=lnpincReg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14, data=rural_reg))


female_urban_reg <- filter(female_urban, !is.infinite(lnpinc) & !is.na(lnpinc))
female_rural_reg <- filter(female_rural, !is.infinite(lnpinc) & !is.na(lnpinc))
male_urban_reg <- filter(male_urban, !is.infinite(lnpinc) & !is.na(lnpinc))
male_rural_reg <- filter(male_rural, !is.infinite(lnpinc) & !is.na(lnpinc))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=female_urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=female_rural_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=male_urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + y10 + y12 + y14, data=male_rural_reg))


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14, data = combined)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.77145 -0.19579  0.00195  0.20434  1.03019 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     0.2432395  0.0040420  60.179  < 2e-16 ***
postsecondary   0.4045173  0.0038999 103.725  < 2e-16 ***
seniorsecondary 0.2750863  0.0035838  76.758  < 2e-16 ***
juniorsecondary 0.2211359  0.0030754  71.905  < 2e-16 ***
primary         0.1165209  0.0033930  34.342  < 2e-16 ***
gender          0.0583887  0.0016172  36.105  < 2e-16 ***
urban           0.0051817  0.0008091   6.404 1.52e-10 ***
y10             0.0630220  0.0036022  17.496  < 2e-16 ***
y12             0.0054965  0.0037955   1.448    0.148    
y14             0.0349121  0.0039403   8.860  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.257


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = urban)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.73505 -0.20376  0.00499  0.20412  1.03628 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     0.232220   0.006038  38.460  < 2e-16 ***
postsecondary   0.394456   0.005413  72.875  < 2e-16 ***
seniorsecondary 0.248885   0.005266  47.260  < 2e-16 ***
juniorsecondary 0.197336   0.004923  40.086  < 2e-16 ***
primary         0.103780   0.005667  18.314  < 2e-16 ***
gender          0.067601   0.002359  28.651  < 2e-16 ***
y10             0.043237   0.004950   8.736  < 2e-16 ***
y12             0.011402   0.005179   2.202   0.0277 *  
y14             0.030583   0.005322   5.746 9.23e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.259 on 30496 degrees of freedom
Multiple R-squared:  0.1946,	Adjusted R-squared:  


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = rural)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.7283 -0.2115 -0.0009  0.2166  1.0945 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.2879958  0.0059436  48.455  < 2e-16 ***
postsecondary    0.3232725  0.0079591  40.617  < 2e-16 ***
seniorsecondary  0.2409509  0.0058254  41.362  < 2e-16 ***
juniorsecondary  0.2114866  0.0042801  49.412  < 2e-16 ***
primary          0.1171572  0.0044544  26.302  < 2e-16 ***
gender           0.0679099  0.0024125  28.150  < 2e-16 ***
y10              0.0619160  0.0055613  11.133  < 2e-16 ***
y12             -0.0009048  0.0059090  -0.153    0.878    
y14              0.0418971  0.0063115   6.638 3.23e-11 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.266 on 27805 degrees of freedom
Multiple R-squared:  0.1494,	Adjusted R-


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14, data = lnpincReg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.7598 -0.3159  0.2362  0.5488  2.8301 

Coefficients:
                 Estimate Std. Error  t value Pr(>|t|)    
(Intercept)     -1.756753   0.014104 -124.561  < 2e-16 ***
postsecondary    1.109395   0.013608   81.526  < 2e-16 ***
seniorsecondary  0.856994   0.012505   68.532  < 2e-16 ***
juniorsecondary  0.744771   0.010731   69.405  < 2e-16 ***
primary          0.430039   0.011839   36.323  < 2e-16 ***
gender           0.165196   0.005643   29.275  < 2e-16 ***
urban            0.022351   0.002823    7.917 2.48e-15 ***
y10              0.144145   0.012569   11.468  < 2e-16 ***
y12             -0.027868   0.013244   -2.104   0.0354 *  
y14              0.070125   0.013749    5.100 3.40e-07 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.7951 -0.3283  0.2534  0.5598  2.9971 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.868052   0.021174 -88.226  < 2e-16 ***
postsecondary    1.169590   0.018981  61.619  < 2e-16 ***
seniorsecondary  0.880941   0.018467  47.703  < 2e-16 ***
juniorsecondary  0.776174   0.017263  44.962  < 2e-16 ***
primary          0.452089   0.019871  22.751  < 2e-16 ***
gender           0.179042   0.008274  21.639  < 2e-16 ***
y10              0.104480   0.017357   6.020 1.77e-09 ***
y12              0.000941   0.018160   0.052 0.958677    
y14              0.065685   0.018665   3.519 0.000433 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.9083 on 30496 degrees of freedom
Multiple R-squared:  0.143,	Adjusted R-squa


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14, data = rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.3908 -0.3163  0.2591  0.5730  2.8186 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.553247   0.020394 -76.161  < 2e-16 ***
postsecondary    0.847606   0.027310  31.036  < 2e-16 ***
seniorsecondary  0.658214   0.019989  32.929  < 2e-16 ***
juniorsecondary  0.613825   0.014686  41.796  < 2e-16 ***
primary          0.371485   0.015285  24.305  < 2e-16 ***
gender           0.202821   0.008278  24.501  < 2e-16 ***
y10              0.114567   0.019083   6.004 1.95e-09 ***
y12             -0.059819   0.020276  -2.950  0.00318 ** 
y14              0.068882   0.021657   3.181  0.00147 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.9127 on 27805 degrees of freedom
Multiple R-squared:  0.1066,	Adjusted R-squ


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = female_urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-8.4650 -0.3546  0.2707  0.5819  1.7356 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.70981    0.03094 -55.268   <2e-16 ***
postsecondary    1.11275    0.02888  38.532   <2e-16 ***
seniorsecondary  0.79484    0.02821  28.176   <2e-16 ***
juniorsecondary  0.75208    0.02658  28.296   <2e-16 ***
primary          0.36708    0.02988  12.286   <2e-16 ***
y10              0.05206    0.02391   2.178   0.0294 *  
y12             -0.02767    0.02499  -1.107   0.2682    
y14              0.01992    0.02581   0.772   0.4403    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.9308 on 16296 degrees of freedom
Multiple R-squared:  0.1026,	Adjusted R-squared:  0.1022 
F-statistic: 266.1 on 7 and 16296 DF,  p-value: < 2.2e


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = female_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.5325 -0.3436  0.2858  0.6166  1.5103 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.43126    0.02765 -51.769  < 2e-16 ***
postsecondary    0.80894    0.03767  21.473  < 2e-16 ***
seniorsecondary  0.67050    0.02661  25.199  < 2e-16 ***
juniorsecondary  0.62134    0.02052  30.280  < 2e-16 ***
primary          0.34986    0.02159  16.205  < 2e-16 ***
y10              0.05249    0.02528   2.077  0.03786 *  
y12             -0.08739    0.02662  -3.283  0.00103 ** 
y14              0.02392    0.02842   0.842  0.40004    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.9348 on 16334 degrees of freedom
Multiple R-squared:  0.06822,	Adjusted R-squared:  0.06782 
F-statistic: 170.8 on 7 and 16334 DF,  p-value: < 2.


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = male_urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.2163 -0.3198  0.2561  0.5477  1.7641 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.787210   0.029521 -60.540  < 2e-16 ***
postsecondary    1.197089   0.025978  46.081  < 2e-16 ***
seniorsecondary  0.928264   0.025204  36.830  < 2e-16 ***
juniorsecondary  0.746330   0.023421  31.865  < 2e-16 ***
primary          0.473865   0.027589  17.176  < 2e-16 ***
y10              0.151875   0.025881   5.868 4.51e-09 ***
y12              0.005864   0.027131   0.216 0.828888    
y14              0.106324   0.027699   3.839 0.000124 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.904 on 14130 degrees of freedom
Multiple R-squared:  0.1482,	Adjusted R-squared:  0.1478 
F-statistic: 351.1 on 7 and 14130 DF,  p-value: 


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14, data = male_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.9587 -0.3083  0.2746  0.6062  1.4407 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.39992    0.03130 -44.727  < 2e-16 ***
postsecondary    0.86910    0.04181  20.788  < 2e-16 ***
seniorsecondary  0.60195    0.03336  18.041  < 2e-16 ***
juniorsecondary  0.56356    0.02292  24.590  < 2e-16 ***
primary          0.34535    0.02322  14.872  < 2e-16 ***
y10              0.15995    0.03057   5.232 1.71e-07 ***
y12             -0.04184    0.03304  -1.266    0.205    
y14              0.14148    0.03518   4.021 5.82e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.9247 on 11386 degrees of freedom
Multiple R-squared:  0.07499,	Adjusted R-squared:  0.07442 
F-statistic: 131.9 on 7 and 11386 DF,  p-value: < 2.2e

In [95]:
index <- c("combined", "urban", "rural", "female_urban", "female_rural", "male_urban", "male_rural")
mean_lnpinc <- c(mean(lnpincReg$lnpinc), mean(urban_reg$lnpinc), mean(rural_reg$lnpinc), mean(female_urban_reg$lnpinc), mean(female_rural_reg$lnpinc), mean(male_urban_reg$lnpinc), mean(male_rural_reg$lnpinc))
median_lnpinc <- c(median(lnpincReg$lnpinc), median(urban_reg$lnpinc), median(rural_reg$lnpinc), median(female_urban_reg$lnpinc), median(female_rural_reg$lnpinc), median(male_urban_reg$lnpinc), median(male_rural_reg$lnpinc))
var_lnpinc <- c(var(lnpincReg$lnpinc), var(urban_reg$lnpinc), var(rural_reg$lnpinc), var(female_urban_reg$lnpinc), var(female_rural_reg$lnpinc), var(male_urban_reg$lnpinc), var(male_rural_reg$lnpinc))
kurtosis_lnpinc <- c(kurtosis(lnpincReg$lnpinc), kurtosis(urban_reg$lnpinc), kurtosis(rural_reg$lnpinc), kurtosis(female_urban_reg$lnpinc), kurtosis(female_rural_reg$lnpinc), kurtosis(male_urban_reg$lnpinc), kurtosis(male_rural_reg$lnpinc))
skew_lnpinc <- c(skewness(lnpincReg$lnpinc), skewness(urban_reg$lnpinc), skewness(rural_reg$lnpinc), skewness(female_urban_reg$lnpinc), skewness(female_rural_reg$lnpinc), skewness(male_urban_reg$lnpinc), skewness(male_rural_reg$lnpinc))

lnpinc <- data.frame("subsamples"=index, "mean" = mean_lnpinc, "median" = median_lnpinc, "var" = var_lnpinc, "kurtosis" = kurtosis_lnpinc, "skew" = skew_lnpinc)
lnpinc$stdev <- sqrt(lnpinc$var)

In [96]:
print("Lnpinc descriptive statistics")
print(lnpinc)

[1] "Lnpinc descriptive statistics"
    subsamples       mean     median       var kurtosis      skew     stdev
1     combined -0.9932873 -0.6929958 0.9476583 4.521023 -1.822567 0.9734774
2        urban -0.9956874 -0.6816732 0.9622872 4.659814 -1.846591 0.9809624
3        rural -0.9902010 -0.6893012 0.9321413 4.440378 -1.809082 0.9654747
4 female_urban -0.9957098 -0.6830242 0.9650912 4.995448 -1.877762 0.9823906
5 female_rural -0.9906684 -0.6931472 0.9374414 4.777774 -1.844421 0.9682156
6   male_urban -0.9955266 -0.7035754 0.9589197 4.369069 -1.817201 0.9792444
7   male_rural -0.9892730 -0.6775680 0.9239143 4.029081 -1.763341 0.9612046


In [94]:
print(nrow(filter(combined, y10 == 1 & !is.na(pinc))))
print(nrow(filter(combined, y12 == 1 & !is.na(pinc))))
print(nrow(filter(combined, y14 == 1 & !is.na(pinc))))
print(nrow(filter(combined, y16 == 1 & !is.na(pinc))))

[1] 25034
[1] 15272
[1] 12401
[1] 6605


### Regression for pinc and lnpinc, with province dummies

In [102]:
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + urban + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + Others, data=combined))
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + Others, data=urban))
summary(lm(pinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + Others, data=rural))

summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + urban + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + Others, data=lnpincReg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + Others, data=urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary + gender + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + Others, data=rural_reg))

summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary
 + y10 + y12 + y14
 + Liaoning + Guangdong + Gansu + Others, data = female_urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary
 + y10 + y12 + y14
 + Liaoning + Guangdong + Gansu + Others, data = female_rural_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary
 + y10 + y12 + y14
 + Liaoning + Guangdong + Gansu + Others, data = male_urban_reg))
summary(lm(lnpinc ~  postsecondary + seniorsecondary + juniorsecondary + primary
 + y10 + y12 + y14
 + Liaoning + Guangdong + Gansu + Others, data = male_rural_reg))


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14 + Liaoning + Guangdong + 
    Gansu + Others, data = combined)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.9010 -0.1894  0.0061  0.1929  1.0778 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.4069579  0.0054855  74.188  < 2e-16 ***
postsecondary    0.3860141  0.0038563 100.100  < 2e-16 ***
seniorsecondary  0.2617895  0.0035341  74.075  < 2e-16 ***
juniorsecondary  0.2109397  0.0030391  69.409  < 2e-16 ***
primary          0.1112692  0.0033412  33.302  < 2e-16 ***
gender           0.0618615  0.0015914  38.872  < 2e-16 ***
urban            0.0012267  0.0008002   1.533   0.1253    
y10              0.0477934  0.0036086  13.244  < 2e-16 ***
y12             -0.0092762  0.0038006  -2.441   0.0147 *  
y14              0.0161992  0.0039396   4.112 3.93e-05 ***
Liaoning        -0.1447790  0.0049421 -29.295  < 2e-


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14 + Liaoning + Guangdong + 
    Gansu + Others, data = urban)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.83986 -0.19743  0.00693  0.19770  0.95689 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.369604   0.007324  50.466   <2e-16 ***
postsecondary    0.385552   0.005338  72.223   <2e-16 ***
seniorsecondary  0.242853   0.005182  46.864   <2e-16 ***
juniorsecondary  0.193394   0.004848  39.889   <2e-16 ***
primary          0.102495   0.005571  18.400   <2e-16 ***
gender           0.071545   0.002321  30.819   <2e-16 ***
y10              0.015619   0.004976   3.139   0.0017 ** 
y12             -0.015051   0.005199  -2.895   0.0038 ** 
y14              0.005252   0.005332   0.985   0.3246    
Liaoning        -0.120229   0.006042 -19.899   <2e-16 ***
Guangdong       -0.085290   0.005948 -14.340   <2e-16 ***
Gansu 


Call:
lm(formula = pinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14 + Liaoning + Guangdong + 
    Gansu + Others, data = rural)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.77823 -0.20720  0.00132  0.20789  1.08160 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.480209   0.010486  45.794  < 2e-16 ***
postsecondary    0.315624   0.007882  40.044  < 2e-16 ***
seniorsecondary  0.237518   0.005766  41.192  < 2e-16 ***
juniorsecondary  0.205940   0.004256  48.391  < 2e-16 ***
primary          0.114348   0.004423  25.852  < 2e-16 ***
gender           0.067875   0.002391  28.390  < 2e-16 ***
y10              0.061557   0.005598  10.995  < 2e-16 ***
y12             -0.000541   0.005953  -0.091    0.928    
y14              0.036638   0.006338   5.781 7.52e-09 ***
Liaoning        -0.201623   0.010263 -19.646  < 2e-16 ***
Guangdong       -0.176381   0.010330 -17.075  < 2e-16 ***
Gansu 


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + urban + y10 + y12 + y14 + Liaoning + Guangdong + 
    Gansu + Others, data = lnpincReg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.6892 -0.3127  0.2147  0.5441  2.7848 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.310790   0.019256 -68.070  < 2e-16 ***
postsecondary    1.056498   0.013537  78.045  < 2e-16 ***
seniorsecondary  0.818046   0.012406  65.939  < 2e-16 ***
juniorsecondary  0.713846   0.010668  66.912  < 2e-16 ***
primary          0.412845   0.011729  35.199  < 2e-16 ***
gender           0.174668   0.005587  31.266  < 2e-16 ***
urban            0.010632   0.002809   3.785 0.000154 ***
y10              0.105064   0.012668   8.294  < 2e-16 ***
y12             -0.065404   0.013342  -4.902  9.5e-07 ***
y14              0.020550   0.013830   1.486 0.137308    
Liaoning        -0.386602   0.017349 -22.284  < 2e-16 ***
Gu


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14 + Liaoning + Guangdong + 
    Gansu + Others, data = urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.6886 -0.3283  0.2321  0.5506  2.8823 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.47489    0.02584 -57.081  < 2e-16 ***
postsecondary    1.14103    0.01883  60.584  < 2e-16 ***
seniorsecondary  0.86178    0.01828  47.137  < 2e-16 ***
juniorsecondary  0.76218    0.01710  44.559  < 2e-16 ***
primary          0.44651    0.01965  22.720  < 2e-16 ***
gender           0.19051    0.00819  23.260  < 2e-16 ***
y10              0.01872    0.01756   1.066    0.286    
y12             -0.08220    0.01834  -4.481 7.45e-06 ***
y14             -0.01421    0.01881  -0.756    0.450    
Liaoning        -0.28763    0.02132 -13.493  < 2e-16 ***
Guangdong       -0.22680    0.02098 -10.808  < 2e-16 ***
Gansu           -0.370


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + gender + y10 + y12 + y14 + Liaoning + Guangdong + 
    Gansu + Others, data = rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.4153 -0.3278  0.2451  0.5792  2.8540 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.045616   0.036112 -28.955  < 2e-16 ***
postsecondary    0.827718   0.027143  30.495  < 2e-16 ***
seniorsecondary  0.648404   0.019857  32.654  < 2e-16 ***
juniorsecondary  0.599268   0.014656  40.890  < 2e-16 ***
primary          0.364008   0.015232  23.897  < 2e-16 ***
gender           0.202605   0.008233  24.608  < 2e-16 ***
y10              0.117451   0.019280   6.092 1.13e-09 ***
y12             -0.054234   0.020500  -2.645  0.00816 ** 
y14              0.058650   0.021827   2.687  0.00721 ** 
Liaoning        -0.566709   0.035342 -16.035  < 2e-16 ***
Guangdong       -0.429490   0.035573 -12.073  < 2e-16 ***
Gansu     


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + 
    Others, data = female_urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-8.3706 -0.3529  0.2607  0.5816  1.7290 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.35793    0.03780 -35.929  < 2e-16 ***
postsecondary    1.08022    0.02876  37.561  < 2e-16 ***
seniorsecondary  0.77416    0.02803  27.620  < 2e-16 ***
juniorsecondary  0.73867    0.02642  27.963  < 2e-16 ***
primary          0.36332    0.02967  12.247  < 2e-16 ***
y10             -0.01957    0.02427  -0.806 0.420037    
y12             -0.09263    0.02528  -3.664 0.000249 ***
y14             -0.04386    0.02607  -1.682 0.092532 .  
Liaoning        -0.26233    0.03047  -8.609  < 2e-16 ***
Guangdong       -0.23374    0.02994  -7.807 6.20e-15 ***
Gansu           -0.27335    0.04248  -6.434 1.28e-10 ***
Others          -0.34882


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + 
    Others, data = female_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.5403 -0.3217  0.2620  0.6112  1.6174 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.94054    0.05004 -18.797  < 2e-16 ***
postsecondary    0.79694    0.03747  21.266  < 2e-16 ***
seniorsecondary  0.65764    0.02648  24.832  < 2e-16 ***
juniorsecondary  0.60404    0.02050  29.467  < 2e-16 ***
primary          0.33693    0.02154  15.640  < 2e-16 ***
y10              0.05871    0.02557   2.296  0.02167 *  
y12             -0.07905    0.02695  -2.934  0.00336 ** 
y14              0.01975    0.02868   0.689  0.49107    
Liaoning        -0.50085    0.04875 -10.273  < 2e-16 ***
Guangdong       -0.45340    0.04931  -9.195  < 2e-16 ***
Gansu           -0.61694    0.04484 -13.757  < 2e-16 ***
Others          -0.47845


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + 
    Others, data = male_urban_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.0852 -0.3123  0.2127  0.5414  1.8283 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -1.346686   0.036392 -37.005  < 2e-16 ***
postsecondary    1.175898   0.025678  45.794  < 2e-16 ***
seniorsecondary  0.914203   0.024867  36.763  < 2e-16 ***
juniorsecondary  0.732369   0.023145  31.642  < 2e-16 ***
primary          0.465664   0.027177  17.135  < 2e-16 ***
y10              0.049429   0.026072   1.896  0.05800 .  
y12             -0.100198   0.027356  -3.663  0.00025 ***
y14              0.005686   0.027859   0.204  0.83828    
Liaoning        -0.284512   0.030516  -9.323  < 2e-16 ***
Guangdong       -0.215106   0.030302  -7.099 1.32e-12 ***
Gansu           -0.458081   0.043895 -10.436  < 2e-16 ***
Others        


Call:
lm(formula = lnpinc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + 
    Others, data = male_rural_reg)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.9724 -0.3096  0.2502  0.5893  1.6069 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     -0.85184    0.05457 -15.610  < 2e-16 ***
postsecondary    0.83438    0.04148  20.115  < 2e-16 ***
seniorsecondary  0.59417    0.03306  17.971  < 2e-16 ***
juniorsecondary  0.54209    0.02286  23.714  < 2e-16 ***
primary          0.33895    0.02309  14.678  < 2e-16 ***
y10              0.16739    0.03079   5.437 5.54e-08 ***
y12             -0.03399    0.03326  -1.022 0.306721    
y14              0.12755    0.03535   3.608 0.000309 ***
Liaoning        -0.65174    0.05354 -12.174  < 2e-16 ***
Guangdong       -0.43655    0.05374  -8.123 5.01e-16 ***
Gansu           -0.73036    0.04882 -14.961  < 2e-16 ***
Others          -0.53443  

### Run urban dummy with no regional and regional with no urban on combined sample

In [90]:
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary
 + y10 + y12 + y14 + urban, data = lnincReg))
summary(lm(lninc ~  postsecondary + seniorsecondary + juniorsecondary + primary
 + y10 + y12 + y14
 + Liaoning + Guangdong + Gansu + Others, data = lnincReg))


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14 + urban, data = lnincReg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.7962  -0.6694   0.2345   0.9589   5.3579 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      8.560426   0.025345 337.757   <2e-16 ***
postsecondary    2.229323   0.023622  94.375   <2e-16 ***
seniorsecondary  1.680655   0.021700  77.449   <2e-16 ***
juniorsecondary  1.486528   0.018597  79.935   <2e-16 ***
primary          0.871957   0.020577  42.375   <2e-16 ***
y10             -0.970859   0.022982 -42.244   <2e-16 ***
y12             -0.415650   0.024015 -17.308   <2e-16 ***
y14             -1.491930   0.024890 -59.942   <2e-16 ***
urban            0.006481   0.004889   1.326    0.185    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.552 on 58471 degrees of freedom
Multiple R-squared:  0.2544,	Adjusted


Call:
lm(formula = lninc ~ postsecondary + seniorsecondary + juniorsecondary + 
    primary + y10 + y12 + y14 + Liaoning + Guangdong + Gansu + 
    Others, data = lnincReg)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.7400  -0.6560   0.2098   0.9503   5.4076 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      9.33440    0.03395  274.92   <2e-16 ***
postsecondary    2.13810    0.02348   91.06   <2e-16 ***
seniorsecondary  1.61390    0.02151   75.03   <2e-16 ***
juniorsecondary  1.43675    0.01849   77.70   <2e-16 ***
primary          0.84742    0.02039   41.56   <2e-16 ***
y10             -1.03814    0.02310  -44.93   <2e-16 ***
y12             -0.47953    0.02415  -19.86   <2e-16 ***
y14             -1.57095    0.02492  -63.03   <2e-16 ***
Liaoning        -0.69094    0.02997  -23.05   <2e-16 ***
Guangdong       -0.52419    0.02982  -17.58   <2e-16 ***
Gansu           -1.01640    0.02969  -34.23   <2e-16 ***
Others          -0.7324